# Data Preprocessing

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Player Valuations dataset

In [21]:
player_valuations = pd.read_csv('../../data/preprocessed_valuations.csv')
player_valuations.drop(columns='Unnamed: 0', inplace=True)
player_valuations.head()

,name,last_season,average_value
0,A.J. Soares,2016.0,2.187500e+05
1,AJ Leitch-Smith,2017.0,1.214286e+05
2,Aaron Appindangoyé,2023.0,8.333333e+05
3,Aaron Bastiaans,2020.0,1.656250e+05
4,Aaron Boupendza,2020.0,4.266667e+06


In [22]:
# Global historical inflation rate. Min year is 2012
inflation_rates = {
    2012: 0.0373,
    2013: 0.0262,
    2014: 0.0235,
    2015: 0.0143,
    2016: 0.0155,
    2017: 0.0219,
    2018: 0.0244,
    2019: 0.0221,
    2020: 0.0193,
    2021: 0.0348,
    2022: 0.0827,
}

def adjust_for_inflation(value, year):
    adjusted_value = value
    for y in range(int(year) + 1, max(inflation_rates.keys()) + 1):
        adjusted_value *= (1 + inflation_rates[y])
    return adjusted_value

if 'adjusted_value' not in player_valuations.columns:
    player_valuations['adjusted_value'] = player_valuations.apply(lambda row: adjust_for_inflation(row['average_value'], row['last_season']), axis=1)
    player_valuations.drop(columns = 'average_value', inplace=True)
    


In [23]:
player_valuations

,name,last_season,adjusted_value
0,A.J. Soares,2016.0,2.672920e+05
1,AJ Leitch-Smith,2017.0,1.451946e+05
2,Aaron Appindangoyé,2023.0,8.333333e+05
3,Aaron Bastiaans,2020.0,1.855626e+05
4,Aaron Boupendza,2020.0,4.780279e+06
...,...,...,...
28686,Ümit Kurt,2016.0,1.043508e+06
28687,Ümit Türker,2016.0,6.109532e+04
28688,Ümran Zambak,2019.0,5.710006e+04
28689,Üstün Bilgi,2013.0,2.064856e+05


In [25]:
num_columns = ['last_season', 'adjusted_value']
num_data = player_valuations[num_columns]
scaler = StandardScaler().fit(num_data)
scaled_valuations = scaler.transform(num_data)

# Replace original data with scaled data 
player_valuations[num_columns] = scaled_valuations   
player_valuations

,name,last_season,adjusted_value
0,A.J. Soares,-0.752289,-0.333430
1,AJ Leitch-Smith,-0.479310,-0.362391
2,Aaron Appindangoyé,1.158566,-0.199166
3,Aaron Bastiaans,0.339628,-0.352816
4,Aaron Boupendza,0.339628,0.737044
...,...,...,...
28686,Ümit Kurt,-0.752289,-0.149313
28687,Ümit Türker,-0.752289,-0.382339
28688,Ümran Zambak,0.066649,-0.383287
28689,Üstün Bilgi,-1.571227,-0.347853


### Player skill dataset

In [6]:
player_ratings = pd.read_csv('G:\My Drive\Github\siads593-M1\data\FUT_player_data.csv')
pd.set_option('display.max_columns', None)
player_ratings.head()

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,...,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed
0,18949,54231,Kylian Mbappé,182,73,24,73,16,18,16,...,88.0,99.0,87.0,73.0,NaN,NaN,NaN,NaN,NaN,99
1,18981,54251,Karim Benzema,185,81,35,607,350,18,164,...,99.0,99.0,99.0,90.0,NaN,NaN,NaN,NaN,NaN,97
2,18982,54249,Zinedine Zidane,185,77,51,112658,2118,18,171,...,87.0,94.0,92.0,83.0,NaN,NaN,NaN,NaN,NaN,92
3,18730,54005,Pelé,173,70,82,112658,2118,54,153,...,90.0,91.0,78.0,61.0,NaN,NaN,NaN,NaN,NaN,96
4,19001,54277,Robert Lewandowski,185,81,35,241,53,37,164,...,99.0,97.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,97


In [10]:
# List of numerical columns
numerical_cols = [
    'height', 'weight', 'age', 'overallRating', 'pace', 'shooting',
    'passing', 'dribbling', 'defending', 'physicality',
    'pace_acceleration', 'pace_sprintSpeed', 'shooting_positioning',
    'shooting_finishing', 'shooting_shotPower', 'shooting_longShots',
    'shooting_volleys', 'shooting_penalties', 'passing_vision',
    'passing_crossing', 'passing_freeKickAccuracy', 'passing_shortPassing',
    'passing_longPassing', 'passing_curve', 'dribbling_agility',
    'dribbling_balance', 'dribbling_reactions', 'dribbling_ballControl',
    'dribbling_dribbling', 'dribbling_composure', 'defending_interceptions',
    'defending_headingAccuracy', 'defending_standingTackle',
    'defending_slidingTackle', 'defending_defenseAwareness',
    'phsyicality_jumping', 'physicality_stamina', 'physicality_strength',
    'physicality_aggression', 'goalkeeper_diving', 'goalkeeper_handling',
    'goalkeeper_kicking', 'goalkeeper_positioning', 'goalkeeper_reflexes',
    'goalkeeper_speed'
]

# List of categorical columns
categorical_cols = [
    'club', 'league', 'nation', 'rarity', 'position', 'foot',
    'attackWorkRate', 'defenseWorkRate', 'cardColor'
]

# Handle missing values for numerical columns with mean of that column
for col in numerical_cols:
    player_ratings[col].fillna(player_ratings[col].mean(), inplace=True)

# Handle missing values for categorical columns with mode (most frequent value) of that column
for col in categorical_cols:
    player_ratings[col].fillna(player_ratings[col].mode()[0], inplace=True)

# Standardize numerical columns
scaler = StandardScaler()
player_ratings[numerical_cols] = scaler.fit_transform(player_ratings[numerical_cols])

# One-hot encode categorical columns
player_ratings_scaled = pd.get_dummies(player_ratings, columns=categorical_cols)
player_ratings_scaled

id  futbin_id                name    height    weight       age  \
0      18949      54231       Kylian Mbappé  0.069935  1.521727 -0.441774   
1      18981      54251       Karim Benzema  0.346944  1.756129  0.756160   
2      18982      54249     Zinedine Zidane  0.346944  1.638928  2.498610   
3      18730      54005                Pelé -0.761091  1.433825  5.874606   
4      19001      54277  Robert Lewandowski  0.346944  1.756129  0.756160   
...      ...        ...                 ...       ...       ...       ...   
19030  14789      47568        Jiahui Zhang  0.069935 -0.617199 -0.877387   
19031  14790      47577           Ziye Zhao -0.114737 -0.617199 -0.986290   
19032  15077      47296     Antonio D'Silva  0.069935 -0.617199 -0.550677   
19033  13807      47579           Junjie Wu  0.623952 -0.617199 -0.877387   
19034  15099      47044       Devansh Dabas  0.069935 -0.617199 -0.659580   

       overallRating      pace  shooting   passing  dribbling  defending  \
0           3.006927  1.798925  2.238685  1.951703   2.170112  -0.236559   
1           3.006927  1.686473  2.287935  2.007656   2.110459   0.001856   
2           3.006927  1.405342  2.140185  2.343375   2.050807   1.766128   
3           3.006927  1.630247  2.189435  2.063609   2.170112   0.526369   
4           3.006927  1.686473  2.287935  1.951703   2.050807   0.144905   
...              ...       ...       ...       ...        ...        ...   
19030      -2.361396 -0.731254 -0.765562 -0.622143  -0.991478  -0.331925   
19031      -2.361396 -0.225218 -0.223812 -0.957862  -0.931825  -1.237902   
19032      -2.361396 -2.699170 -1.947560 -2.244785  -2.065225  -1.857781   
19033      -2.361396 -0.675027 -1.356561 -1.573347  -1.707310  -0.093510   
19034      -2.361396 -2.193135 -1.947560 -2.132878  -2.184531  -1.905464   

       physicality  pace_acceleration  pace_sprintSpeed  shooting_positioning  \
0         1.480001           1.938032          1.951400              2.042407   
1         2.141186           1.812729          1.824902              2.042407   
2         1.678356           1.562121          1.508655              1.901442   
3         0.884935           1.750077          1.761652              1.995418   
4         2.273422           1.875380          1.824902              2.042407   
...            ...                ...               ...                   ...   
19030    -0.900263          -1.257214         -0.641819             -0.166036   
19031    -1.363093          -0.066828         -0.515320             -0.729893   
19032    -2.156514          -3.199422         -2.982041             -2.327490   
19033    -0.503553          -0.881303         -0.768317             -1.152787   
19034    -1.693685          -2.698207         -2.286299             -2.421466   

       shooting_finishing  shooting_shotPower  shooting_longShots  \
0                2.227204            2.423348            1.996066   
1                2.227204            2.423348            2.137732   
2                2.041350            2.161580            2.232176   
3                2.227204            2.161580            2.043288   
4                2.227204            2.423348            2.232176   
...                   ...                 ...                 ...   
19030           -0.978779           -0.717872           -0.884484   
19031           -0.095972            0.132875           -0.978928   
19032           -2.000976           -1.764946           -1.970592   
19033           -1.489878           -1.895830           -1.262261   
19034           -1.954513           -1.764946           -2.065037   

       shooting_volleys  shooting_penalties  passing_vision  passing_crossing  \
0              2.370765            2.334378        2.294032          1.859971   
1              2.569602            2.561278        2.545640          1.550041   
2              2.470184            2.391103        2.545640          2.324865   
3              2.420474            2.391103   